<a href="https://colab.research.google.com/github/tattwamasiray/BigData-Challenge/blob/main/Home_Sales_starter_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install findspark


In [2]:
import sys
print(sys.version)


3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]


In [3]:
!pip install pyspark


In [4]:
import os

# Set the SPARK_HOME environment variable
os.environ["SPARK_HOME"] = "/usr/local/lib/python3.10/dist-packages/pyspark"

# Initialize findspark
import findspark
findspark.init()



In [5]:
# Import findspark and initialise.
import findspark
findspark.init()

In [6]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [7]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [8]:
# 2. Create a temporary view of the DataFrame.
# Create a temporary view of the DataFrame
# Import packages

import time
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
data_df = spark.read.csv("file://" + SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)

# 2. Create a temporary view of the DataFrame.
data_df.createOrReplaceTempView("home_sales")
# Show the first few rows of the DataFrame
data_df.show()





+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [9]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
# Run a Spark SQL query to calculate the average price for a four-bedroom house sold in each year
average_price_query = """
    SELECT year(date) AS sales_year, ROUND(AVG(price), 2) AS avg_price
    FROM home_sales
    WHERE bedrooms = 4
    GROUP BY year(date)
    ORDER BY sales_year
"""

# Execute the query and show the result
average_price_result = spark.sql(average_price_query)
average_price_result.show()


+----------+---------+
|sales_year|avg_price|
+----------+---------+
|      2019| 300263.7|
|      2020|298353.78|
|      2021|301819.44|
|      2022|296363.88|
+----------+---------+



In [10]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?

average_price_query = """
    SELECT date_built, ROUND(AVG(price), 2) AS avg_price
    FROM home_sales
    WHERE bedrooms = 3 AND bathrooms = 3
    GROUP BY date_built
    ORDER BY date_built
"""

# Execute the query and show the result
average_price_result = spark.sql(average_price_query)
average_price_result.show()

+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|292859.62|
|      2011|291117.47|
|      2012|293683.19|
|      2013|295962.27|
|      2014|290852.27|
|      2015| 288770.3|
|      2016|290555.07|
|      2017|292676.79|
+----------+---------+



In [11]:
#  5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

# Run a Spark SQL query to calculate the average price of homes built in each year with specific criteria
average_price_query = """
    SELECT date_built, ROUND(AVG(price), 2) AS avg_price
    FROM home_sales
    WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
    GROUP BY date_built
    ORDER BY date_built
"""

# Execute the query and show the result
average_price_result = spark.sql(average_price_query)
average_price_result.show()


+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|285010.22|
|      2011|276553.81|
|      2012|307539.97|
|      2013|303676.79|
|      2014|298264.72|
|      2015|297609.97|
|      2016| 293965.1|
|      2017|280317.58|
+----------+---------+



In [12]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

import time

# Start measuring time
start_time = time.time()

# Run a Spark SQL query to calculate the average price and view rating for homes with price >= $350,000
average_price_view_query = """
    SELECT AVG(view) AS avg_view
    FROM home_sales
    WHERE price >= 350000
"""

# Execute the query and collect the result
average_price_view_result = spark.sql(average_price_view_query).collect()

# Calculate runtime
runtime = time.time() - start_time

# Display the average view rating rounded to two decimal places
print("Average View Rating:", round(average_price_view_result[0]['avg_view'], 2))

# Display the runtime
print("Runtime:", runtime, "seconds")


Average View Rating: 32.26
Runtime: 0.8045403957366943 seconds


In [13]:
# 7. Cache the the temporary table home_sales.
spark.table("home_sales").cache()


DataFrame[id: string, date: date, date_built: int, price: int, bedrooms: int, bathrooms: int, sqft_living: int, sqft_lot: int, floors: int, waterfront: int, view: int]

In [14]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [15]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

from pyspark.sql.functions import avg
import time

# Calculate the average price for homes with a view rating greater than or equal to $350,000
start_time_cached = time.time()

# Run the query using the cached data
result_cached = spark.sql("SELECT view, ROUND(AVG(price), 2) AS avg_price FROM home_sales WHERE price >= 350000 GROUP BY view")

# Show the result
result_cached.show()

print("--- Cached Query Runtime: %s seconds ---" % (time.time() - start_time_cached))

# Uncached Query Runtime
start_time_uncached = time.time()

# Run the query using the original data without caching
result_uncached = spark.sql("SELECT view, ROUND(AVG(price), 2) AS avg_price FROM home_sales WHERE price >= 350000 GROUP BY view")

# Show the result
result_uncached.show()

print("--- Uncached Query Runtime: %s seconds ---" % (time.time() - start_time_uncached))


+----+----------+
|view| avg_price|
+----+----------+
|  31| 399856.95|
|  85|1056336.74|
|  65| 736679.93|
|  53|  755214.8|
|  78|1080649.37|
|  34| 401419.75|
|  81|1053472.79|
|  28| 402124.62|
|  76|1058802.78|
|  26| 401506.97|
|  27| 399537.66|
|  44| 400598.05|
|  12| 401501.32|
|  91|1137372.73|
|  22| 402022.68|
|  93|1026006.06|
|  47|  398447.5|
|   1| 401044.25|
|  52| 733780.26|
|  13| 398917.98|
+----+----------+
only showing top 20 rows

--- Cached Query Runtime: 2.0604448318481445 seconds ---
+----+----------+
|view| avg_price|
+----+----------+
|  31| 399856.95|
|  85|1056336.74|
|  65| 736679.93|
|  53|  755214.8|
|  78|1080649.37|
|  34| 401419.75|
|  81|1053472.79|
|  28| 402124.62|
|  76|1058802.78|
|  26| 401506.97|
|  27| 399537.66|
|  44| 400598.05|
|  12| 401501.32|
|  91|1137372.73|
|  22| 402022.68|
|  93|1026006.06|
|  47|  398447.5|
|   1| 401044.25|
|  52| 733780.26|
|  13| 398917.98|
+----+----------+
only showing top 20 rows

--- Uncached Query Runtime:

In [22]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
# Import packages
from pyspark.sql import SparkSession
import time
# Define the local path to save Parquet files (use a new directory or specify an existing directory to overwrite)
output_parquet_path = "/new_path/to/output/directory"

# Write the DataFrame in Parquet format partitioned by "date_built"
data_df.write.mode("overwrite").partitionBy("date_built").parquet(output_parquet_path)




In [23]:
# 11. Read the formatted parquet data.
# Define the path to the directory containing the Parquet files
parquet_directory = "/new_path/to/output/directory"


# Read the Parquet formatted data into a DataFrame
parquet_data_df = spark.read.parquet(parquet_directory)

# Show the first few rows of the DataFrame
parquet_data_df.show()


+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3|      2015|
|c797ca12-52cd-4b1...|2019-06-08|288650|       2|        3|       2100|   10419|     2|         0|   7|      2015|
|0cfe57f3-28c2-472...|2019-10-04|308313|       3|        3|       1960|    9453|     2|         0|   2|      2015|
|d715f295-2fbf-4e9...|2021-05-17|391574|       3|        2|       1635|    8040|     2|         0|  10|      2015|
|a18515a2-86f3-46b...|2022-02-18|419543|       3|        2|       1642|   12826|

In [24]:
# 12. Create a temporary table for the parquet data.
# Create a temporary view of the DataFrame
parquet_data_df.createOrReplaceTempView("parquet_home_sales")



In [27]:
# 13. Run the query that filters out the view ratings with average price of greater than or eqaul to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.
from pyspark.sql import functions as F
import time

# Start the timer for uncached query
start_time_uncached = time.time()

# Run the query on uncached data
uncached_result = spark.sql("""
    SELECT ROUND(AVG(price), 2) AS avg_price
    FROM home_sales
    WHERE price >= 350000
""")

# Calculate the uncached runtime
uncached_runtime = time.time() - start_time_uncached

# Show the result
uncached_result.show()

# Start the timer for parquet query
start_time_parquet = time.time()

# Run the query on parquet data
parquet_result = spark.sql("""
    SELECT ROUND(AVG(price), 2) AS avg_price
    FROM parquet_home_sales
    WHERE price >= 350000
""")

# Calculate the parquet runtime
parquet_runtime = time.time() - start_time_parquet

# Show the result
parquet_result.show()

# Calculate and print the runtime difference
print("--- Difference in Runtimes (Uncached - Parquet): %s seconds ---" % (uncached_runtime - parquet_runtime))


+---------+
|avg_price|
+---------+
|473796.26|
+---------+

+---------+
|avg_price|
+---------+
|473796.26|
+---------+

--- Difference in Runtimes (Uncached - Parquet): -0.006417751312255859 seconds ---


In [28]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")


In [29]:
# 15. Check if the home_sales is no longer cached
is_cached = spark.catalog.isCached("home_sales")
print("Is home_sales cached?", is_cached)



Is home_sales cached? False
